In [80]:
import matplotlib.pyplot as plt
%matplotlib inline
import time
from matplotlib import animation
import math
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import cm
import colorcet as cc


In [86]:
def normalize(x, xmin, xmax):
    return (x - xmin) / (xmax - xmin)

def plot_vor_colored(L, x, ax, cmap):
    """
    Plot the Voronoi.

    Takes in a set of cell locs (x), tiles these 9-fold, plots the full voronoi, then crops to the field-of-view

    :param x: Cell locations (nc x 2)
    :param ax: matplotlib axis
    """

#     L = self.L
    grid_x, grid_y = np.mgrid[-1:2, -1:2]
    grid_x[0, 0], grid_x[1, 1] = grid_x[1, 1], grid_x[0, 0]
    grid_y[0, 0], grid_y[1, 1] = grid_y[1, 1], grid_y[0, 0]
    y = np.vstack(
        [
            x + np.array([i * L, j * L])
            for i, j in np.array([grid_x.ravel(), grid_y.ravel()]).T
        ]
    )

    cmap_print = np.tile(cmap.T, 9).T
    bleed = 0.1
    cmap_print = cmap_print[
        (y < L * (1 + bleed)).all(axis=1) + (y > -L * bleed).all(axis=1)
    ]
    y = y[(y < L * (1 + bleed)).all(axis=1) + (y > -L * bleed).all(axis=1)]
    regions, vertices = voronoi_finite_polygons_2d(Voronoi(y))

    ax.set(aspect=1, xlim=(0, L), ylim=(0, L))
    patches = []
    for i, region in enumerate(regions):
        patches.append(
            Polygon(
                vertices[region],
                True,
                facecolor=cmap_print[i],
                edgecolor="white",
                alpha=0.5,
            )
        )

    p = PatchCollection(patches, match_original=True)
    # p.set_array(c_types_print)
    ax.add_collection(p)

def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.

    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.

    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.

    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

def animate_GRN(
    t_span,
    x_save,
    E_save,
    L,
    file_name=None,
    dir_name="plots",
    n_frames=100,
    fps=15,
    cmap="CET_L8",
    print_updates=True,
):
    """
    Animate the simulation, saving to an mp4 file.

    Parameters
    ----------
    n_frames : int
        Number of frames to animate. Spaced evenly throughout **x_save**

    file_name : str
        Name of the file. If **None** given, generates file-name based on the time of simulation

    dir_name: str
        Directory name to save the plot.


    """
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    fig = plt.figure()
    ax1 = fig.add_subplot(1, 1, 1)

    n_frames = min(n_frames, x_save.shape[0])
    skip = math.floor(x_save.shape[0] / n_frames)

    E_sample = E_save[::skip]
    E_min, E_max = E_sample.min(), E_sample.max()

    def anim(i):
        ax1.cla()
        colors = cc.cm[cmap](normalize(E_sample[i], E_min, E_max))
        plot_vor_colored(L, x_save[skip * i], ax1, colors)
        ax1.set(aspect=1, xlim=(0, L), ylim=(0, L))
        ax1.set_title(f"time = {t_span[skip * i]:.3f}")

    Writer = animation.writers["ffmpeg"]
    writer = Writer(fps=fps, bitrate=1800)
    if file_name is None:
        file_name = "animation_%d" % time.time()
    an = animation.FuncAnimation(fig, anim, frames=n_frames)

    fname = "%s/%s.mp4" % (dir_name, file_name)
    if print_updates:
        print(f"Saving to {fname}")

    an.save(fname, writer=writer, dpi=264)
    plt.close()


In [63]:
# # vor_path = "/home/ubuntu/git/active_vertex"
# vor_path = 'C:\\Users\\Pranav\\git\\active_vertex'

# sys.path.append(vor_path)
# import voronoi_model.voronoi_model_periodic as avm

In [88]:
%load_ext blackcellmagic

<hr>

In [68]:
# Read metadata of all sims in batch
data_dir = "C:/Users/Pranav/git/evomorph/2020-09-09_avm_phase_sims_examples/"
df = pd.read_csv(os.path.join(data_dir, "metadata_full.csv"), index_col = 0)

# Extract parameter spaces
p_space, v_space, rep_space = (
    np.unique(df.p0.values),
    np.unique(df.v0.values),
    np.unique(df.replicate.values),
)

# Get data 
example_files = [os.path.split(f)[-1] for f in glob(data_dir + "*rep?.*", )]

# Pick lattice to simulate
lax_fname = example_files[0]
lax_fname

In [69]:
# Extract data and metadata
x_save = np.load(os.path.join(data_dir, lax_fname))
E_save = np.load(os.path.join(data_dir, lax_fname[:-4] + "_Esave.npy"))
metadata = df.loc[df["data_fname"] == lax_fname,]
metadata = metadata.iloc[0, :].to_dict()

# Get GRN time-span
t_span = np.linspace(metadata["t0"], metadata["tmax"], metadata["n_t"])

# Get window size
L = metadata["L"]

In [70]:
# vor = avm.Tissue()
# vor.L = L
# vor.x_save = x_save
# vor.E_save = E_save
# vor.GRN_t_span = t_span

In [87]:
animate_GRN(
    t_span,
    x_save,
    E_save,
    L,
    n_frames=100, 
    cmap="kgy", 
    print_updates=True
)

Saving to plots/animation_1599872910.mp4
